# メモ書き
## key(項目)
### jpcrp_cor:CompanyNameCoverPage
企業名
### jppfs_cor:Assets
総資産
### jpcrp_cor:NetSalesSummaryOfBusinessResults
売上高:製品や商品など有形物を販売する場合
### jpcrp_cor:OperatingRevenue1SummaryOfBusinessResults
営業収益:無形のサービスや手数料などを提供した場合
### 例外(jpcrp030000-asr_E\*\*\*\*\*-***:NAME)
定義済みではない個社毎に独自に開示しているデータ，E\*\*\*\*\* は会社番号でNAMEに項目名が入る．  
asrはPublicDoc内の.xbrlファイル名に含まれる謎の文字列

## context_ref(詳細)
### FilingDateInstant
企業名のコンテキスト(詳細不明)
### CurrentYearInstant_NonConsolidatedMember
CurrentYearInstant:今年度  
NonConsolidatedMember:単体情報(親会社-持ち株会社のみ) ，ない場合は連結情報（子会社含む，ホールディングスと名前につくことが多い）  
Instantは総資産のみに使われていると思われる
#### CurrentYearDuration_NonConsolidatedMember
Duration:期間 は 代n期を示す  
Current:今期, Prior1:前期, Prior2:前々期, 以降さかのぼるごとにPriorに続く数値が増える

In [1]:
#!usr/bin/env python3
# -*- coding: utf-8 -*-

In [33]:
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
from bs4 import BeautifulSoup
import fnmatch
import csv
import os

In [3]:
# 全xbrlファイルパスを取得する(ディレクトリを含んでしまう)
def find_all_xbrl(directory):
    for root, dirs, files in os.walk(directory):
        yield root
        for file in files:
            if fnmatch.fnmatch(file, '*.xbrl'):
                yield os.path.join(root, file)

In [4]:
# 全ファイルパスを取得
all_file_path = []
for f in find_all_xbrl("..\extracted_xbrl"):
    all_file_path.append(f)

In [10]:
# xbrl形式の入るパスのみ取得
xbrl_file_path = []
for i in all_file_path:
    # PublicDoc内のxbrlファイルはasrという文字列を含む
    if fnmatch.fnmatch(i, '*asr*.xbrl'):
        xbrl_file_path.append(i)
print("get " + str(len(xbrl_file_path)) + " file paths")

get 533 file paths


In [11]:
# パーサの初期化
parser = EdinetXbrlParser()

In [12]:
# xbrlファイルを取得してpythonで扱えるオブジェクトに格納
edinet_xbrl_obj = parser.parse_file(xbrl_file_path[246]) # 24カネカ 240住友理工 246東レ

In [13]:
# 企業名を取得
print(xbrl_file_path[246])
key = "jpcrp_cor:CompanyNameCoverPage"
context_ref = "FilingDateInstant"
company_name = edinet_xbrl_obj.get_data_by_context_ref(key, context_ref).get_value()
print("Company: " + company_name)

..\extracted_xbrl\Xbrl_Search_20181129_114356\S100D9YD\XBRL\PublicDoc\jpcrp030000-asr-001_E00873-000_2018-03-31_01_2018-06-26.xbrl
Company: 東レ株式会社


In [14]:
# 今年度の有形固定資産を取得
print(xbrl_file_path[246])
key = "jpcrp_cor:PropertyPlantAndEquipmentInformationForEachRegionTextBlock"
context_ref = "CurrentYearDuration"
tangivleFixedAssets = edinet_xbrl_obj.get_data_by_context_ref(key, context_ref).get_value()
print("tangivleFixedAssets: " + tangivleFixedAssets)
type(tangivleFixedAssets)

..\extracted_xbrl\Xbrl_Search_20181129_114356\S100D9YD\XBRL\PublicDoc\jpcrp030000-asr-001_E00873-000_2018-03-31_01_2018-06-26.xbrl
kaigai: 
<p class="smt_text6" style="padding-left:18pt;">(2) 有形固定資産</p>
<div class="tbld" style="margin-left:0pt;margin-right:0pt;margin-top:0pt;margin-bottom:0pt;">
<table cellspacing="0" cellpadding="0" style="border-collapse:collapse;border:solid 0pt #000000;width:360pt;">
<colgroup>
<col style="width:60pt;min-width:60pt;"/>
<col style="width:60pt;min-width:60pt;"/>
<col style="width:60pt;min-width:60pt;"/>
<col style="width:60pt;min-width:60pt;"/>
<col style="width:60pt;min-width:60pt;"/>
<col style="width:60pt;min-width:60pt;"/>
</colgroup>
<tr style="height:15pt">
<td valign="middle" rowspan="2" style="vertical-align:middle;border-top-style:solid;border-top-width:0.75pt;border-bottom-style:solid;border-bottom-width:0.75pt;border-left-style:solid;border-left-width:0.75pt;border-right-style:solid;border-right-width:0.75pt;">
<p class="smt_tblC">日本<br/>(

bs4.element.NavigableString

In [47]:
# 邪魔な<br/>タグを除去
tangivleFixedAssets = str(kaigai).replace("<br/>", "")

# 有形固定資産の該当箇所の抽出
soup = BeautifulSoup(tangivleFixedAssets)
fixedAssetsList = soup.find_all("p", class_="smt_tblC")
for i in fixedAssetsList:
    print(i.string)

日本(百万円)
アジア
欧米他
合計(百万円)
韓国(百万円)
その他(百万円)
米国(百万円)
欧州他(百万円)


C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [56]:
# 総資産を取得
key = "jppfs_cor:Assets"
context_ref = "CurrentYearInstant_NonConsolidatedMember"
current_year_assets = edinet_xbrl_obj.get_data_by_context_ref(key, context_ref).get_value()
print("Assets: " + current_year_assets + " JPN")

Assets: 12768925000 JPN


In [58]:
# 売上高を取得
key = "jpcrp_cor:NetSalesSummaryOfBusinessResults"
context_ref = "CurrentYearDuration_NonConsolidatedMember"
current_year_duration = edinet_xbrl_obj.get_data_by_context_ref(key, context_ref).get_value()
print("Sales: " + current_year_duration + " JPN")

Sales: 6312682000 JPN


In [32]:
# 売上が1000億円以上の企業を取得
target_company = []
target_sales = []

sales_context_ref = "CurrentYearDuration_NonConsolidatedMember" # 単体情報(親会社のみ)

company_key = "jpcrp_cor:CompanyNameCoverPage"
company_context_ref = "FilingDateInstant"

for i in xbrl_file_path:
    sales_key = "jpcrp_cor:NetSalesSummaryOfBusinessResults"
    
    # 指定オブジェクトの取得
    edinet_xbrl_obj = parser.parse_file(i)
    sales = edinet_xbrl_obj.get_data_by_context_ref(sales_key, sales_context_ref)
    
    # 売上高が存在しないなら，営業収益を取得
    if sales is None:
        sales_key = "jpcrp_cor:OperatingRevenue1SummaryOfBusinessResults"
        sales = edinet_xbrl_obj.get_data_by_context_ref(sales_key, sales_context_ref)
    
    # 売上高，営業収益ともにないなら除外
    if sales is None:
        pass
    else:
        sales = sales.get_value()
    
    if sales is None:
        pass
    else:
        # 1000億円以上の売上ならリストに追加
        if int(sales) >= 100000000000:
            company = edinet_xbrl_obj.get_data_by_context_ref(company_key, company_context_ref).get_value()
            target_company.append(company)
            target_sales.append(sales)

print(target_company)
print(target_sales)

['レンゴー株式会社', '日本新薬株式会社', '日本コークス工業株式会社', '株式会社ツムラ', '株式会社カネカ', '宇部興産株式会社', '武田薬品工業株式会社', '日本製紙株式会社', '出光興産株式会社', '信越化学工業株式会社', '関西ペイント株式会社', '大王製紙株式会社', '日立化成株式会社', '出光興産株式会社', '日本コークス工業株式会社', '旭化成株式会社', 'エア・ウォーター株式会社', '積水化学工業株式会社', '日油株式会社', '株式会社エフピコ', '三菱製紙株式会社', '日本化薬株式会社', '北越紀州製紙株式会社', '日産化学工業株式会社', '富士石油株式会社', '東ソー株式会社', '日本ゼオン株式会社', '大日精化工業株式会社', '持田製薬株式会社', '株式会社コーセー', '日本ゼオン株式会社', 'アイカ工業株式会社', '株式会社ＡＤＥＫＡ', '日東電工株式会社', '住友理工株式会社', '東レ株式会社', '三洋化成工業株式会社', 'ダイキョーニシカワ株式会社', '田辺三菱製薬株式会社', '小野薬品工業株式会社', '株式会社トクヤマ', '株式会社ダイセル', 'ハリマ化成グループ株式会社', '三菱瓦斯化学株式会社', '東洋紡株式会社', 'グンゼ株式会社', '参天製薬株式会社', '沢井製薬株式会社', '三井化学株式会社', '小林製薬株式会社', '東洋ゴム工業株式会社', '横浜ゴム株式会社', 'ＤＩＣ株式会社', '昭和電工株式会社', 'ライオン株式会社', '住友ゴム工業株式会社', 'コクヨ株式会社', '小林製薬株式会社', '久光製薬株式会社', '大日本住友製薬株式会社', '株式会社日本触媒', '日医工株式会社', 'トヨタ紡織株式会社', '第一三共株式会社', 'アステラス製薬株式会社', '住友化学株式会社', 'イビデン株式会社', '大陽日酸株式会社', 'ＪＳＲ株式会社', 'コニカミノルタ株式会社', '豊田合成株式会社', 'エーザイ株式会社', '帝人株式会社', '塩野義製薬株式会社', 'デンカ株式会社', 'アステラス製薬株式会社', '関西ペイント株式会社', '大王製紙株式会社', '信越化学工業株式会社'

In [54]:
# 売上が1000億円以上の企業を取得
target_company = []
target_sales = []

sales_context_ref = "CurrentYearDuration_NonConsolidatedMember" # 単体情報(親会社のみ)

company_key = "jpcrp_cor:CompanyNameCoverPage"
company_context_ref = "FilingDateInstant"

for i in xbrl_file_path:
    sales_key = "jpcrp_cor:NetSalesSummaryOfBusinessResults"
    
    # 指定オブジェクトの取得
    edinet_xbrl_obj = parser.parse_file(i)
    sales = edinet_xbrl_obj.get_data_by_context_ref(sales_key, sales_context_ref) # 売上高の取得
    
    # 売上高が存在しないなら，営業収益を取得
    if sales is None:
        sales_key = "jpcrp_cor:OperatingRevenue1SummaryOfBusinessResults"
        sales = edinet_xbrl_obj.get_data_by_context_ref(sales_key, sales_context_ref)
    
    # 売上高，営業収益ともにないなら除外
    if sales is None:
        pass
    else:
        sales = sales.get_value()
    
    if sales is None:
        pass
    else:
        # 1000億円以上の売上ならリストに追加
        if int(sales) >= 100000000000:
            company = edinet_xbrl_obj.get_data_by_context_ref(company_key, company_context_ref).get_value()
            
            tfa_key = "jpcrp_cor:PropertyPlantAndEquipmentInformationForEachRegionTextBlock"
            tfa_context_ref = "CurrentYearDuration"
            tangivleFixedAssets = edinet_xbrl_obj.get_data_by_context_ref(tfa_key, tfa_context_ref)
            
            if (tangivleFixedAssets is None):
                pass
            else:
                tangivleFixedAssets = tangivleFixedAssets.get_value()
                
                # 邪魔な<br/>タグを除去して，有形固定資産の該当箇所の抽出
                soup = BeautifulSoup(str(tangivleFixedAssets).replace("<br/>", ""))
                fixedAssetsList = soup.find_all("p", class_="smt_tblC")
                for i in fixedAssetsList:
                    if fnmatch.fnmatch(i.string, '*米国*') or fnmatch.fnmatch(i.string, '*北米*'):
                        target_company.append(company)
                        target_sales.append(sales)

print(target_company)
print(target_sales)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


['株式会社カネカ', '日本製紙株式会社', '旭化成株式会社', '三菱製紙株式会社', '東レ株式会社', '東洋ゴム工業株式会社', '東洋ゴム工業株式会社', '久光製薬株式会社', 'イビデン株式会社', '株式会社資生堂', '株式会社クラレ', '株式会社クラレ']
['293016000000', '611735000000', '621875000000', '118445000000', '591664000000', '218678000000', '218678000000', '112391000000', '128869000000', '220407000000', '242657000000', '242657000000']


In [36]:
with open('company_list.csv', 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerow(target_company)